*Inizialmente in questo notebook lavoreremo con alcuni file definiti "...moviesTweetsPre" che contengono i tweet scaricati prima dell'inizio dell'evento e con alcuni file definiti "...MoviesTweets" che invece contengono i tweet scaricati durante l'evento, a seguito dell'unione di questi file lavoreremo con il dataset "complete_movies"*

## 1. Importiamo le librerie necessarie per la fase di pre-processing

In [1]:
import pandas as pd 
import re
from numpy import nan
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
! pip install emoji --upgrade

Requirement already up-to-date: emoji in /home/monica/snap/jupyter/common/lib/python3.7/site-packages (1.5.0)


Utilizziamo la funzione set option della libreria Pandas che ci consente di visualizzare il testo completo

In [2]:
pd.set_option('display.max_colwidth', None)

## 2. Importiamo i  file json contenenti i tweet scaricati

Tre file sono relativi ai tweet scaricati nelle ore precedenti all'evento 

In [3]:
M_preMovies =  pd.read_json(r"tweet/M_moviesTweetsPre.txt") #4687 rows × 7 columns
C_preMovies =  pd.read_json(r"tweet/C_moviesTweetsPre.txt") #4729 rows × 7 columns
B_preMovies =  pd.read_json(r"tweet/B_moviesTweetsPre.txt") #4888 rows × 7 columns

Concateniamo i 3 dataset ottenendo il dataset completo "tot_premovies"

In [4]:
tot_premovies = pd.concat([M_preMovies,C_preMovies,B_preMovies]) #14289 rows × 7 columns

Ulteriori tre file sono relativi ai tweet scaricati durante l'evento

In [5]:
M_Movies =  pd.read_json(r"tweet/M_MoviesTweets") #144320 rows × 7 columns
B_Movies =  pd.read_json(r"tweet/B_MoviesTweets") #145678 rows × 7 columns
C_Movies =  pd.read_json(r"tweet/C_MoviesTweets") #147567 rows × 7 columns

Concateniamo i 3 dataset ottenendo un dataset completo "tot_movies"

In [6]:
tot_movies = pd.concat([M_Movies,C_Movies,B_Movies]) #437436 rows × 7 columns

Creiamo ora un dataframe "complete_movies" contenente tutti i tweet acquisiti prima dell'evento e durante l'evento e ne correggiamo l'ora 

In [7]:
complete_movies = pd.concat([tot_movies,tot_premovies], ignore_index=True)
complete_movies['date'] = pd.DatetimeIndex(complete_movies['date']) + timedelta(hours=-7)

Mettiamo in ordine i tweet in base all'ora di pubblicazione ed eliminiamo i tweet duplicati tramite la funzione drop duplicates

In [8]:
complete_movies.sort_values(by = ['date'], inplace=True, ignore_index=True)
complete_movies.drop_duplicates(subset = ["user_id", "date", "text"], ignore_index= True, inplace=True) #153666 rows × 7 columns

## 3. Analisi descrittiva 
1. Numero di utenti singoli e di tweet pubblicati durante la serata
2. Numero user bot (tweet pubblicati allo stesso secondo) 
3. Numero tweet e retweet per utente pre eliminazione dei bot 
4. Numero tweet e retweet per utente post eliminazione dei bot 
5. Numero tweet e retweet per film
6. Numero user bot (tweet complessivi per utente > 50) 

### 3.1 Numero di utenti singoli e di tweet pubblicati durante la serata

Contiamo il numero di singoli utenti presenti nel dataset "complete_movies" (81199) ed i rispettivi tweet pubblicati e salviamo in un file csv il dataset "user_twitter"

In [10]:
user_twitter = complete_movies.groupby(["screen_name"]).count()
user_twitter.to_csv(r"results/user_twitter_Mpre.csv")

### 3.2 Numero user bot (tweet pubblicati allo stesso secondo) 

Dopodiché contiamo quanti di questi utenti hanno pubblicato tweet alla stessa ora e allo stesso secondo "user_bot_movies" , raggruppando sia per nome che per data. Affinché si trovino gli utenti che hanno pubblicato più tweet allo stesso secondo imponiamo la condizione: "id" > 1, tali utenti rappresentano per noi dei bot che verranno eliminati successivamente "user_bot"

In [12]:
user_bot_movies = complete_movies.groupby(["screen_name", "date"]).count()
user_bot= user_bot_movies[user_bot_movies['_id']>1] 
unique_users_bot = user_bot.groupby(["screen_name"]).count()
user_bot_movies.reset_index(inplace = True)
user_bot.reset_index(inplace = True)
unique_users_bot.reset_index(inplace = True)
unique_users_bot.to_csv(r"results/nome_utenti_BOT.csv")

In [13]:
unique_users_bot

,screen_name,_id,user_id,username,text,hashtags
0,AdonicaB,1,1,1,1,1
1,Ambivertti,1,1,1,1,1
2,Anonymous13268,1,1,1,1,1
3,BoukhetAbdou,1,1,1,1,1
4,Cyrus_balls,1,1,1,1,1
5,DogPawDK,1,1,1,1,1
6,Dwenn_Colas_ht,1,1,1,1,1
7,GlobbalConnect,1,1,1,1,1
8,JulioPinillaZ,1,1,1,1,1
9,MKSafdar,1,1,1,1,1


Con la seguente funzione contiamo quanti tweet sono stati pubblicati dagli user bot durante la serata, il totale risulta pari a 375, di questi 95 sono stati pubblicati nello stesso secondo

In [14]:
i = 0 
for bot in set(unique_users_bot["screen_name"]):
    for utente in complete_movies["screen_name"]:
        if bot == utente: 
            i = i+1

In [15]:
i

375

### 3.3 Numero tweet e retweet per utente pre eliminazione dei bot 

Creiamo il dataset "retweet" che contiene i soli retweet presenti nel dataset "complete_movies" e salviamo in "value_number_rt" il numero complessivo di retweet in tutto il dataset "complete_movies" prima che avvenga l'eliminazione dei bot

In [16]:
complete_movies['text'] = complete_movies['text'].str.lower()

In [17]:
retweet = complete_movies[complete_movies['text'].str.contains('\srt @|^rt @', regex=True)] 
value_number_rt = retweet.count()

In [18]:
value_number_rt

_id            108065
date           108065
user_id        108065
username       108065
screen_name    108065
text           108065
hashtags       108065
dtype: int64

Creiamo successivamente due dataset "users_RT" e "user_TW" dove per ogni utente è presente rispettivamente il numero di retweet e di tweet pubblicati. Infine "count_TW_RT" rappresenta l'unione dei due dataset precedenti, in tale dataset è presente: il numero di retweet (NumberRetweet) e di tweet per utente (NumberTweet), la percentuale di retweet sul totale dei tweet pubblicati dal singolo utente (RetweetRate) ed infine la percentuale di retweet sul totale dei tweet pubblicati da tutti gli utenti (RetweetRate_tot). 

In [19]:
users_RT = retweet.groupby('screen_name').count()
users_TW = complete_movies.groupby('screen_name').count()
count_TW_RT = users_TW.merge(users_RT, how='left', left_on=['screen_name'], right_on=['screen_name'])
count_TW_RT  = count_TW_RT[['user_id_x','user_id_y']]
count_TW_RT['RetweetRate'] = (count_TW_RT['user_id_y']/count_TW_RT['user_id_x'])*100
count_TW_RT['RetweetRate_tot'] = (count_TW_RT['user_id_y']/153716)*100
count_TW_RT["diff"] = count_TW_RT['user_id_x'] - count_TW_RT['user_id_y']
count_TW_RT.reset_index(inplace=True)
count_TW_RT.rename(columns={"user_id_x":"NumberTweet", "user_id_y":"NumberRetweet", "screen_name":"UserName"}, inplace = True)

In [20]:
count_TW_RT.to_csv(r"results/user_twitter_Mpre.csv")

# Procediamo con l'eliminazione degli utenti bot 

Prepariamo la colonna "text" e carichiamo il dataset "trackMovies" che contiene la lista dei nomi dei film. Tale dataset ci consentirà di runnare la funzione "trova", nello specifico quest'ultima valuta se all'interno del tweet viene menzionato il film, una volta trovato il il titolo lo inserisce in una nuova colonna "film", in caso contrario tale cella rimane vuota. Tale funzione consente successivamente di mantenere nel dataset solo quei tweet in cui effettivamente è stato nominato l'attore o l'attrice, scartando pertanto tutti i tweet che sono stati scaricati erroneamente. 

In [21]:
complete_movies['text'] = complete_movies['text'].str.replace("\n"," ", regex=True)
complete_movies['text'] = complete_movies['text'].str.replace("#","", regex=True)
complete_movies['text'] = complete_movies['text'].str.replace("http\S+","", regex=True)
complete_movies['text'] = complete_movies['text'].str.lower()

In [22]:
trackMovies = pd.read_csv(r"dataset/trackMovies.csv", sep = ",")
trackMovies['TitleComplete'] = trackMovies['title']

In [23]:
trackMovies['title'] = trackMovies['title'].str.replace("\s?the\s+|^a\s+|\s?movie\s+|\s?film\s?|\s?and\s+|\s?in\s+|\s?vs\.\s+|\s+of\s+", ' ')
trackMovies['title'] = trackMovies['title'].str.replace("\s+a\s+", " ")
trackMovies['title'] = trackMovies['title'].str.replace("  ", " ")
trackMovies['title'] = trackMovies['title'].str.replace("^ ", "")
trackMovies['title'] = trackMovies['title'].str.replace(" $", "")

In [25]:
trackMovies.to_csv(r"results/trackMovies_NC.csv")

In [26]:
def trova (riga):
    c=0
    trovato=[]
    for i, j in zip(trackMovies['title'], trackMovies['keys']):
        d=0
        c=0
        a=i
        lista = i.split()
        lunghezza = len(lista) 
        for k in lista:
            if bool(re.search(f'\s*{k}\s*', riga)) is True:
                c = c + 1
            elif bool(re.search(f'\s*{j}\s*', riga)) is True:
                d = 1 
        if ((lunghezza == 1 & c==1) | d==1):
            trovato+=[a]
            
        elif ((lunghezza == 2 & c==2) | d==1):
            trovato+=[a]
             
        elif ((lunghezza == 3 & c>=2) | d==1):
            trovato+=[a]
        
        elif ((lunghezza == 4 & c>=2) | d==1):
            trovato+=[a]      
    return (trovato)

In [27]:
complete_movies['film'] = complete_movies['text'].apply(trova)

Creiamo un nuovo dataset "tweet_per_movies" tramite la funzione _explode_ che crea una riga per ogni citazione a film che viene fatta all'interno del tweet ed eliminiamo sia i tweet che non hanno citato alcun film sia tutti i tweet associati a un utente bot. I tweet che vengono eliminati perché non citano film sono pari a 34630 (161643 -127013)

In [28]:
tweet_per_movies= complete_movies.explode('film')

In [29]:
tweet_per_movies.dropna(inplace=True)

In [30]:
null_tweet_eliminated = 153716 -127052
null_tweet_eliminated

26664

Eliminiamo ora tutti i tweet pubblicati da utenti bot che hanno tweettato nella stessa data (ora e secondi)

In [31]:
for bot in set(unique_users_bot['screen_name']):
     for utente in tweet_per_movies["screen_name"]:
            if bot == utente: 
                tweet_per_movies = tweet_per_movies.drop(tweet_per_movies["screen_name"].loc[tweet_per_movies["screen_name"]==utente].index)

In [32]:
bot_tweet_eliminated = 127052 -126737 
bot_tweet_eliminated

315

### Assegnamoo ad ogni film presente nella colonna "film" il codice IMDB corrispondente

In [33]:
def codice (riga):
    trovato=[]
    for i, j in zip(trackMovies['title'], trackMovies['IMDB_Id']):
        if riga["film"] == i: 
            return (j)

In [34]:
tweet_per_movies['imdb'] = tweet_per_movies.apply(codice, axis = 1)

In [35]:
tweet_per_movies.to_csv(r"results/tweet_per_movies.csv")

In [36]:
tweet_per_movies.head(50)

,_id,date,user_id,username,screen_name,text,hashtags,film,imdb
0,6085da962ab79c00399db267,2021-04-25 14:09:37,754497085718659072,La Lonely Soul,candelasonidera,rt @madmenpics: mank (2020) oscars,[Oscars],mank,tt10618286
1,6085da982ab79c00399db26b,2021-04-25 14:09:39,1010983127144165376,Rhia ✨ 𝐀𝐟𝐭𝐞𝐫 𝐖𝐞 𝐅𝐞𝐥𝐥,RhiLovesAfter,i’m kinda glad not many ppl are gonna be at the oscars who i wanna see. i also don’t know half the films nominated this year 😅 apart from promising young woman! i usually love award season and stay up to watch the oscars carpet etc. there’s no point this time lmao,[],promising young woman,tt9620292
2,6085da9a2ab79c00399db26f,2021-04-25 14:09:40,186933748,George Roussos,baphometx,rt @apentertainment: oscars watch: glenn close shares how she got into character for her oscar-nominated role as mamaw in hillbillyelegy.…,[HillbillyElegy],hillbilly elegy,tt6772802
3,6085da992ab79c00399db26c,2021-04-25 14:09:40,1339619325578240000,Film Updates,TheFilmUpdates,paul raci is the first nominee to hit the red carpet at the oscars nominated for best supporting actor in sound of metal 🥁,[Oscars],sound metal,tt5363618
5,6085da9f2ab79c00399db274,2021-04-25 14:09:46,1117987430,Matt Ferguson,Matt_The_Ferg,"just finished watching all the oscars best picture, international, animated, documentary, screenplay noms right under the wire!! so much to love, but nomadland is a particularly special film. that one’s gonna stick with me.",[Oscars],nomadland,tt9770150
7,6085daa22ab79c00399db278,2021-04-25 14:09:49,845715754758668288,ka 🥑,oscarsxka,one night in miami 3.5/5 oscars oscarnoms,"[oscars, OscarNoms]",one night miami,tt10612922
10,6085daa42ab79c00399db27f,2021-04-25 14:09:51,1282432923833118720,🧊,neonwarr,rt @thefilmupdates: judas and the black messiah makes oscars history as the first best picture nominee with an all-black producing team: s…,[Oscars],judas black messiah,tt9784798
11,6085daa52ab79c00399db282,2021-04-25 14:09:52,1372767556386189312,˜”*°• Verr || hyped my 📌,appleejuuicee,rt @dylanobrien: really cool breakdown of how this sequence was executed in loveandmonsters courtesy of our vfx supervisor matt sloan (and…,[LoveAndMonsters],love monsters,tt2222042
13,6085daa72ab79c00399db286,2021-04-25 14:09:53,2231267737,buginette🐞,sunofhaiz,rt @dylanobrien: really cool breakdown of how this sequence was executed in loveandmonsters courtesy of our vfx supervisor matt sloan (and…,[LoveAndMonsters],love monsters,tt2222042
14,6085daaa2ab79c00399db288,2021-04-25 14:09:56,905891842620882944,sᴀᴍ,areyouboredofmi,rt @dylanobrien: really cool breakdown of how this sequence was executed in loveandmonsters courtesy of our vfx supervisor matt sloan (and…,[LoveAndMonsters],love monsters,tt2222042


## Creazione dataset numero tweet e retweet per utente post eliminazione dei bot

### 3.4 Numero tweet e retweet per utente post eliminazione dei bot 

Creiamo il dataset "number_rt1" che contiene i soli retweet presenti nel dataset "tweet_per_movies" e salviamo in "tot_rt1" il numero complessivo di retweet in tutto il dataset "tweet_per_movies" (97880).

In [37]:
number_rt1 = tweet_per_movies[tweet_per_movies['text'].str.contains('\srt @|^rt @', regex=True)] 
tot_rt1 = number_rt1.count()

In [38]:
tot_rt1

_id            97680
date           97680
user_id        97680
username       97680
screen_name    97680
text           97680
hashtags       97680
film           97680
imdb           97680
dtype: int64

Creiamo successivamente due dataset "users_RT1" e "users_TW1" dove per ogni utente è presente rispettivamente il numero di retweet e di tweet pubblicati. Infine "count_TW_RT1" rappresenta l'unione dei due dataset precedenti, in tale dataset è presente: il numero di retweet (NumberRetweet) e di tweet per utente (NumberTweet), la percentuale di retweet sul totale dei tweet pubblicati dal singolo utente (RetweetRate) ed infine la percentuale di retweet sul totale dei tweet pubblicati da tutti gli utenti (RetweetRate_tot). 

In [39]:
users_RT1 = number_rt1.groupby(['screen_name', 'date']).count()
users_TW1 = tweet_per_movies.groupby(['screen_name', 'date']).count()
users_RT2 = users_RT1.groupby('screen_name').count()
users_TW2 =users_TW1.groupby('screen_name').count()
count_TW_RT1 = users_TW2.merge(users_RT2, how='left', left_on=['screen_name'], right_on=['screen_name'])
count_TW_RT1  = count_TW_RT1[['user_id_x','user_id_y']]
count_TW_RT1['RetweetRate'] = (count_TW_RT1['user_id_y']/count_TW_RT1['user_id_x'])*100
count_TW_RT1["RetweetRate_tot"] = (count_TW_RT1['user_id_y']/134645*100)
count_TW_RT1["diff"] = count_TW_RT1['user_id_x'] - count_TW_RT1['user_id_y']
count_TW_RT1.reset_index(inplace=True)
count_TW_RT1.rename(columns={"user_id_x":"NumberTweet", "user_id_y":"NumberRetweet","screen_name":"UserName"}, inplace = True)

In [41]:
count_TW_RT1.to_csv(r"results/count_user_movies_posteliminazione.csv")

### 3.5 Numero tweet e retweet per film

Svolgiamo gli stessi procedimenti per i film, ottenendo un nuovo dataset "count_TW_RT_M" che presenta per ogni film il numero di tweet e di retweet in cui è stato menzionato

In [42]:
number_RT_M = number_rt1.groupby('film').count()
number_TW_M = tweet_per_movies.groupby(['film']).count()
count_TW_RT_M = number_RT_M.merge(number_TW_M, how='inner', on='film')
count_TW_RT_M=count_TW_RT_M[['_id_x', '_id_y' ]]
count_TW_RT_M['RetweetRate'] = (count_TW_RT_M['_id_x']/count_TW_RT_M['_id_y'])*100
count_TW_RT_M["RetweetRate_tot"] = (count_TW_RT_M['_id_y']/134645*100)
count_TW_RT_M["diff"] = count_TW_RT_M["_id_y"] - count_TW_RT_M["_id_x"]
count_TW_RT_M.reset_index(inplace=True)
count_TW_RT_M = count_TW_RT_M.rename(columns={"_id_x":"NumberRetweet", "_id_y":"NumberTweet","reg":"ActorName"})
count_TW_RT_M
count_TW_RT_M.to_csv(r"results/count_TW_RT_Movie_post.csv")

### 3.6 Numero user bot (tweet complessivi per utente > 50) 

Valutiamo ora gli utenti che potrebbero essere considerati bot qualora il numero di tweet pubblicati durante la serata sia considerevole. In "user_high_tweets" sono presenti i soli utenti che presentano un numero di tweet superiore a 50, poiché tali utenti non hanno pubblicato nel medesimo secondo e siccome rappresentano solo 28/81199 = 0.03% non si è ritenuto opportuno eliminarli

In [43]:
tweet_per_user = complete_movies.groupby("screen_name").count()
user_high_tweets = tweet_per_user[tweet_per_user["_id"]>50]

## 4. Valutazione utenti con screen_name o username costituiti da soli numeri

In [44]:
complete_movies[complete_movies['screen_name'].str.contains('^\d+$', regex=True)]
complete_movies[complete_movies['username'].str.contains('^\d+$', regex=True)]

,_id,date,user_id,username,screen_name,text,hashtags,film
6167,{'$oid': '608604b52ab79c0039b69233'},2021-04-25 17:09:19,618129055,666,plathist,rt @thefilmupdates: emerald fennell wins best original screenplay at the oscars for promising young woman,[Oscars],[promising young woman]
10891,{'$oid': '608605fc2ab79c003a6b9af7'},2021-04-25 17:14:46,11408872,324,324,rt @variety: oscars: thefather wins best adapted screenplay,"[Oscars, TheFather]",[father]
16228,{'$oid': '608608202ab79c003a6bb784'},2021-04-25 17:23:54,69163268,655321,Alejo_Godoy,rt @oneperfectshot: the father (2020) cinematography by ben smithard written by christopher hampton and florian zeller directed by florian…,[],[father]
18300,{'$oid': '608608b9a7b11b003a59f2d2'},2021-04-25 17:26:27,3208775450,3,Pitchayood,rt @davidehrlich: i love another round because it’s like… what if men were drunk and happy? oscars,[Oscars],[another round]
19066,{'$oid': '608609042ab79c003a6bc565'},2021-04-25 17:27:43,330970436,0308,zno000,rt @mubi: best international feature film: another round oscars,[Oscars],[another round]
...,...,...,...,...,...,...,...,...
146323,{'$oid': '60862fe92ab79c003a6fb97e'},2021-04-25 20:13:39,3208775450,3,Pitchayood,rt @geeksofcolor: congratulations to frances mcdormand on winning the academy award for best actress in a leading role for her performance…,[],[]
146412,{'$oid': '60862feda7b11b003a5deb81'},2021-04-25 20:13:43,954920996024152064,2013,O613_twt,"rt @gma: yuh-jung youn won the academy award for best supporting actress for her performance in ""minari"" at the 2021 oscars on sunday. htt…",[Oscars],[minari]
147996,{'$oid': '6086303d2ab79c003a6fc86a'},2021-04-25 20:15:03,330970436,0308,zno000,rt @mubi: best actress: frances mcdormand (nomadland) oscars,[Oscars],[nomadland]
150305,{'$oid': '6086309f2ab79c003a6fd913'},2021-04-25 20:16:41,254365409,24,Gtheprince16,"rt @theacademy: oscars moment: @hermusicx, @dmile85 and @tiara_thomas win best original song for ""fight for you"" from judas and the black…",[Oscars],[]


## 5. Valutazione utenti con emoji nello username

In [45]:
complete_movies[complete_movies['username'].str.contains("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", regex=True)]

,_id,date,user_id,username,screen_name,text,hashtags,film
11,6085daa52ab79c00399db282,2021-04-25 14:09:52,1372767556386189312,˜”*°• Verr || hyped my 📌,appleejuuicee,rt @dylanobrien: really cool breakdown of how this sequence was executed in loveandmonsters courtesy of our vfx supervisor matt sloan (and…,[LoveAndMonsters],[love monsters]
13,6085daa72ab79c00399db286,2021-04-25 14:09:53,2231267737,buginette🐞,sunofhaiz,rt @dylanobrien: really cool breakdown of how this sequence was executed in loveandmonsters courtesy of our vfx supervisor matt sloan (and…,[LoveAndMonsters],[love monsters]
15,6085daa92ab79c00399db287,2021-04-25 14:09:56,61352170,🤴My Baby & My Princess👸,BubbleRK,rt @pattinsnfilm: appreciation post for tenet soundtrack which should’ve been nominated at the oscars tonight,[],[tenet]
29,6085dabf2ab79c00399db2a2,2021-04-25 14:10:17,1123264993,Teddy🇨🇷🏳️‍🌈🇺🇲,TeddySincero,rt @nbcchicago: oscars 2021 nominee riz ahmed in ‘sound of metal' has already won by breaking stereotypes,[],[sound metal]
33,6085dac22ab79c00399db2a9,2021-04-25 14:10:20,1301906563519721472,dudɑ ψ 📖: Mulherzinhas| six of crows,reggiesummer,rt @thefilmupdates: paul raci is the first nominee to hit the red carpet at the oscars nominated for best supporting actor in sound of me…,[Oscars],[]
...,...,...,...,...,...,...,...,...
153686,{'$oid': '608631362ab79c003a6fefe5'},2021-04-25 20:19:12,109274193,⁺ ◟ 𝘻𝘪𝘮𝘣𝘢𝘻𝘢𝘭𝘢𝘣𝘪𝘮 ⁷🕯,armygl33girl,"rt @risefallnickbck: look people might not have really known there was a live action pinocchio film before the oscars this year, but don't…",[Oscars],[pinocchio]
153694,{'$oid': '608631372ab79c003a6ff001'},2021-04-25 20:19:13,1215075747498348544,나의피로회복제💙찾아삼만리,PickMeUp_LkgFor,rt @variety: oscars: yuh-jung youn wins best supporting actress for minari,"[Oscars, Minari]",[minari]
153697,{'$oid': '608631372ab79c003a6ff00b'},2021-04-25 20:19:13,32216543,Carmen😈,WrigleyCub220,"so proud of @anthonyhopkins for his oscars win for best actor for the father. at 83, he is the oldest to ever win. congratulations sir anthony!",[Oscars],[father]
153705,{'$oid': '608631372ab79c003a6feff9'},2021-04-25 20:19:13,1400952127,みーすけ ブロンズD 帰省 ✨祝T&B 二期✨🐰🐯,meechan178,"rt @imdb: congratulations to anthony hopkins, winner of best performance by an actor in a leading role! thefather 👏🎉 oscars","[TheFather, Oscars]",[father]


## 6. Attraverso il file "trackMovies" costruiamo un dizionario in cui ad ogni film viene associato il numero totale di tweet in cui viene citato

In [46]:
numeroTweetMovies = {}
for title in trackMovies['title']:
    
    numeroTweetMovies[title] = 0

In [47]:
for i,j in zip(trackMovies['title'], trackMovies['TitleComplete'].str.title()):
    somma = tweet_per_movies['film'].str.contains(i, case=False).sum()
    numeroTweetMovies[i] = somma
    numeroTweetMovies[j] = numeroTweetMovies.pop(i)

In [48]:
numeroTweetMovies

{'Borat Subsequent Movie Film': 54,
 'A Shaun The Sheep Movie Farmageddon': 13,
 'Another Round': 7128,
 'Da 5 Bloods': 81,
 'Emma.': 275,
 'The Story Of Fire Saga': 23,
 'Greyhound': 324,
 'Hillbilly Elegy': 310,
 'Judas And The Black Messiah': 12989,
 'Love And Monsters': 1334,
 "Ma Rainey'S Black Bottom": 3919,
 'Mank': 7303,
 'Minari': 25291,
 'Mulan': 391,
 'News Of The Worlds': 1,
 'Nomadland': 22597,
 'One Night In Miami': 591,
 'Onward': 636,
 'Over The Moon': 395,
 'Pieces Of A Woman': 134,
 'Pinocchio': 1249,
 'Promising Young Woman': 6771,
 'Soul': 16649,
 'Sound Of Metal': 11045,
 'Tenet': 4291,
 'The Father': 7169,
 'The Life Ahead': 326,
 'The Midnight Sky': 30,
 'The One And Only Ivan': 21,
 'The Trial Of The Chicago 7': 674,
 'The United States Vs. Billie Holiday': 37,
 'The White Tiger': 74,
 'Wolfwalkers': 2520}

### 6.1 Trasformiamo il dizionario in un dataframe che definiamo "nTweetMovies" in cui ad ogni attore/attrice è associato il numero totale di tweet in cui viene menzionato/a

In [49]:
nTweetMovies = pd.DataFrame.from_dict(numeroTweetMovies, orient='index')
nTweetMovies.reset_index(inplace=True)
nTweetMovies.columns = ['Movies', 'nTweet']
nTweetMovies.sort_values("Movies")

,Movies,nTweet
1,A Shaun The Sheep Movie Farmageddon,13
2,Another Round,7128
0,Borat Subsequent Movie Film,54
3,Da 5 Bloods,81
4,Emma.,275
6,Greyhound,324
7,Hillbilly Elegy,310
8,Judas And The Black Messiah,12989
9,Love And Monsters,1334
10,Ma Rainey'S Black Bottom,3919


In [50]:
nTweetMovies.to_csv(r"results/nTweetMovies.csv")